# Example LePHARE run on an AGN sample

In the minimal photoz run example notebook we demonstrated a run on the COSMOS2020 (Weaver et al. 2022) data set in order to show the most basic LePHARE functionality.

In this notebook we want to walk through a typical use case where the user wishes to run on a new catalogue with a new set of filters.

We will be looking at the same COSMOS data set but only use the ugrizy bands. We will take just these filters from the local auxiliary database. This should demonstrate the basic procedure for updating the configuration parameters and creating an input table in the appropriate format.

The template for this notebook can be downloaded [here](https://github.com/lephare-photoz/lephare/blob/main/docs/notebooks/Typical_use_case.ipynb).

In this notebook we will look at a sample of AGN that are detected in the X-ray by [Marchesi et al. 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...817...34M/abstract) in the COSMOS field. We start with the template and modify it for our example.

We will inspect the best fit SEDs for AGN and compare to a sample of galaxies.

Spectroscopic redhsifts are from [Khostovan et al. 2025](https://arxiv.org/abs/2503.00120). These also provide a broad line sample which can be compared and contrasted to X-ray selected AGN.

We use photometry from HSC-CLAUDS presented in [Sawicki et al. 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.489.5202S/abstract)


In [ ]:
# lephare must be installed if not already
#!pip install lephare 

In [ ]:
import os
import lephare as lp
from astropy.table import Table
import numpy as np
from matplotlib import pylab as plt
import yaml
%load_ext wurlitzer

## Load the example data

In the [documentation example](https://lephare.readthedocs.io/en/latest/notebooks/Typical_use_case.html) we were looking at COSMOS data. Here we are looking at a different data set from CLAUDS discussed above. It actually uses the same filters as some of the COSMOS data so some parts of the configuration are identical.

In [ ]:
input_lp=Table.read("./input_flux_ebv_corrected_agn_sample_20251101.fits")

In [ ]:
input_lp[:5]

In [ ]:
# We have four types of object
for s in np.unique(input_lp['string_input']):
    print(s, np.sum(input_lp['string_input']==s))

### Creating the input table

If we had not provided a table in the correct format we would need to do so. It must be in the following format.

We need to make an astropy table as input. This can be done using the standard column order:
id, flux0, err0, flux1, err1,..., context, zspec, arbitrary_string. A simple example table with two filters might look like this:
|  id | flux_filt1  |  fluxerr_filt1 |  flux_filt2  |  fluxerr_filt2 | context | zspec | string_data |
|---|---|---|---|---|---|---|---|
|  0 | 1.e-31  | 1.e-32  | 1.e-31  | 2.e-32  | 3 | NaN | "This is just a note" |
|  1 | 2.e-31  |  1.e-32 | 1.e-31  | 2.e-32  |3 | 1. | "This has a specz" |
|  2 | 2.e-31 | 1.e-32  | 2.e-31  | 2.e-32  | 2 | NaN| "This context only uses the second filter" |

The context detemermines which bands are used but can be -99 or a numpy.nan. We do not need to have units on the flux columns but LePHARE assumes they are in erg /s /cm**2 / Hz if we are using fluxes. The number of columns must be two times the number of filters plus the four additional columns.

This input table **must use** the standard column ordering to determine column meaning. This odering depends on the filter order in the config FILTER_LIST value. 

## Update the config
We will start with the COSMOS configuration as a basis. We will update the various keywords. We use the default which is shipped with lephare. You could also download the example text file config from [here](https://github.com/lephare-photoz/lephare-data/blob/main/examples/COSMOS.para) or write it completely from scratch.

In [ ]:
config=lp.default_cosmos_config.copy()
name='CLAUDS_EXAMPLE'
bands='ugrizy'

config.update({
    'AUTO_ADAPT': 'YES',
    'CAT_IN': 'bidon',
    'EM_DISPERSION': '1.',
    'ERR_SCALE': '0.02,0.02,0.02,0.02,0.02,0.02', 
    'ERR_FACTOR': '1.5,1.5,1.5,1.5,1.5,1.5', 
    'FILTER_CALIB': '0,0,0,0,0,0',
    'FILTER_FILE': 'filter_lsst',
    'FILTER_LIST': f'cosmos/u_new.pb,hsc/gHSC.pb,hsc/rHSC.pb,hsc/iHSC.pb,hsc/zHSC.pb,hsc/yHSC.pb',
    'GAL_LIB': f'{name}_GAL_BIN',
    'GAL_LIB_IN': f'{name}_GAL_BIN',
    'GAL_LIB_OUT': f'{name}_GAL_MAG',
    'GLB_CONTEXT': np.sum(2**np.arange(len(bands))),
    'INP_TYPE': 'F', # Default is F SExtractor in M, HSCPipe in F
    'MABS_CONTEXT': np.sum(2**np.arange(len(bands))),
    'MABS_REF': '1',
    'MAG_REF':f'2', # The g band
    'NZ_PRIOR':f'2,-1', # the g band
    'QSO_LIB': f'{name}_QSO_PSF_BIN',
    'QSO_LIB_IN': f'{name}_QSO_PSF_BIN',
    'QSO_LIB_OUT': f'{name}_QSO_PSF_MAG',
    'MAG_ABS_QSO': '-30,-20.5',
    'QSO_SED':'sed/QSO/SALVATO2010/SALVATO10.list',
    'SPEC_OUT': 'save_spec',
    'STAR_LIB': f'{name}_STAR_BIN',
    'STAR_LIB_IN': f'{name}_STAR_BIN',
    'STAR_LIB_OUT': f'{name}_STAR_MAG',
    'ZPHOTLIB': f'{name}_STAR_MAG,{name}_GAL_MAG,{name}_QSO_PSF_MAG',
    'Z_STEP': '0.02,0.,6.',
})

star_overrides={}

gal_overrides={
    'MOD_EXTINC':'18,26,26,33,26,33,26,33',
    'EXTINC_LAW':'SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat',  
    'EM_LINES':'EMP_UV',   
}

qso_overrides={
    'MOD_EXTINC':'1,30',  
    'EXTINC_LAW':"SMC_prevot.dat",
    #'EB_V':"0.,0.1,0.2,0.3,0.4", 
    'EB_V': '0.,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.5',
    'EM_LINES':'NO', 
    'MAG_ABS_QSO': '-30,-20.5',
    'QSO_SED':'sed/QSO/SALVATO2010/SALVATO10.list',
    'QSO_LIB': f'{name}_QSO_PSF_BIN',
    'QSO_LIB_IN': f'{name}_QSO_PSF_BIN',
    'QSO_LIB_OUT': f'{name}_QSO_PSF_MAG',
    'ZPHOTLIB': f'{name}_STAR_MAG,{name}_GAL_MAG,{name}_QSO_PSF_MAG'
}

# We might also want to consider a distinct set of AGN configurations with updated SEDs and absolute magnitude priors
agn_overrides={
    'MOD_EXTINC':'1,999',  
    'EXTINC_LAW':"SMC_prevot.dat",#'SMC_prevot.dat', #SB_calzetti.dat',
   # 'EB_V':"0.,0.1,0.2,0.3",
    'EB_V': '0.,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.5',
    'EM_LINES':'NO', 
    'MAG_ABS_QSO': '-24,-8',
   # 'QSO_SED':'sed/QSO/BROWN_TONIMA/Brown_29APR',
    'QSO_SED':'sed/QSO/SALVATO2010/SALVATO10.list',
    'QSO_LIB': f'{name}_AGN_BIN',
    'QSO_LIB_IN': f'{name}_AGN_BIN',
    'QSO_LIB_OUT': f'{name}_AGN_MAG',
    'ZPHOTLIB': f'{name}_STAR_MAG,{name}_GAL_MAG,{name}_AGN_MAG'
}

# Make sure Absolute magnitude priors are set correctly
config.update(qso_overrides)

## Download the required SEDs and additional extinction laws
If one has already cloned the full auxiliary data one does not need to use this functionality.

Here we will need the same set of SEDs and other files required for the COSMOS example so will download those using the automated download functionality.

In [ ]:
# If you want all the files with many different filters and templates etc
# lp.data_retrieval.get_auxiliary_data(clone=False)

In [ ]:
# We need to download the required files
lp.data_retrieval.get_auxiliary_data(
    keymap=config,
    # The additional extinction laws for galaxies are not in the principle config
    # so we must add them to be downloaded:
    additional_files=[
        "ext/SMC_prevot.dat",
        "ext/SB_calzetti.dat",
        "ext/SB_calzetti_bump1.dat",
        "ext/SB_calzetti_bump2.dat",
    ],
)

In [ ]:
for n, f in enumerate(config['FILTER_LIST'].split(',')):
    data = Table.read(f"{lp.LEPHAREDIR}/filt/{f}", format="ascii")
    plt.plot(data[data.colnames[0]], data[data.colnames[1]], label=f)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.xlabel("Wavelength [Angstrom]")
plt.ylabel("Transmission")


In [ ]:
# We need to make the lephare.FilterSvc object to use these filters with lephare
filterLib = lp.FilterSvc.from_keymap(lp.all_types_to_keymap(config))

# These can also be downloaded from the SVO

You can browse names to find your own filters:

https://svo2.cab.inta-csic.es/theory/fps/index.php?mode=browse

When you have the names of all the filters you need you can add them to a yaml file as in the example below:

In [ ]:
# We can download filters automatically from the SVO by making a list of filter names in a yaml file
filters_yaml=yaml.safe_load("""filters:
  trans: 1
  calib: 0
  list:
    - name: svo:CFHT/MegaCam.u
    - name: svo:Subaru/HSC.g
    - name: svo:Subaru/HSC.r
    - name: svo:Subaru/HSC.i
    - name: svo:Subaru/HSC.z
    - name: svo:Subaru/HSC.Y
""")
with open('example_svo_filters.yaml', 'w') as f:
    yaml.dump(filters_yaml, f, sort_keys=False, default_flow_style=False)

In [ ]:
# We use an example yaml file to retrieve the filter names used by the SVO
filterLibSVO = lp.FilterSvc.from_yaml("example_svo_filters.yaml")
filter_output = os.path.join(os.environ["LEPHAREWORK"], "filt", config["FILTER_FILE"])
lp.write_output_filter(filter_output + "_svo.dat", filter_output + "_svo.doc", filterLibSVO)

In [ ]:
f=filterLibSVO[0]
f.name.split('/')[-1]

In [ ]:
# Check the filters are similar
fig = plt.figure(figsize=(15, 8))
for f, fsvo in zip(filterLib, filterLibSVO):
    d = f.data()
    plt.semilogx(d[0], d[1] / d[1].max(),label=f"Local {f.name.split('/')[-1]}")
    dsvo = fsvo.data()
    plt.semilogx(dsvo[0], dsvo[1] / dsvo[1].max(), ".",label=f"SVO {fsvo.name.split('/')[-1]}")
plt.legend()
plt.xlabel('Wavelength [Angstrom]')
plt.ylabel('Relative transmission')

## Run prepare

These are the key preparatory stages that calculate the filters in the LePHARE format, calculate the library of SEDs and finally calculate the library of magnitudes for all the models. The prepare method runs *filter*, *sedtolib*, and *mag_gal* that would be run independently at the command line. These are all explained in detail in the [documentation](https://lephare.readthedocs.io/en/latest/original.html#detailed-lephare-user-manual).

In this example we are running twice with different AGN settings to apply to different AGN samples.

In [ ]:
lp.prepare(config,star_config=star_overrides, gal_config=gal_overrides, qso_config=qso_overrides)
lp.prepare(config,star_config=star_overrides, gal_config=gal_overrides, qso_config=agn_overrides)

## Run process

Finally we run the main fitting process which is equivalent to *zphota* when using the command line. We also need to update some of the config values to make them consistent with the number of filters.

In the particular example we provide there is an additional column with the extended ness that we need to remove prior to running.

In [ ]:
# In this example I added an extracolumn for class star which can help to distinguish between types of AGN
keepcols=[s for s in input_lp.colnames if s != "CLASS_STAR_HSC_I"]
output, photozlist=lp.process(config, input_lp[keepcols],write_outputs=True)
config_agn=config.copy()
config_agn.update(agn_overrides)
output_agn, photozlist_agn=lp.process(config_agn, input_lp[keepcols],write_outputs=True)

## Take a quick look at the output

You can see the main columns present int he output. Some columns regarding phsyical parameters are not present because we have not computed them in a standard run. you can see an example [here](https://lephare.readthedocs.io/en/latest/notebooks/Typical_use_case_physicalParameters.html) for computing them.

In [ ]:
output[:5]

## Investigate the results

Now we want to look at which template set performs best in a given situation. Can the $\chi^2$ for each template set be used to determine which estimate to use for a given object? Can we use the extendedness to choose the template set? Ground based data may not beable to provide a god enough extendedness measure to determine if an object is a Quasar dominated by the AGN emission or a local galaxy with some AGN contribution.

Can you implement some of the key metrics for point estimate performance to compare the various samples?

There are a number of metrics to evaluate the performance of photometric redshift estimates in addition to visual inspection. These can be broadly separated into metrics of point prediction performance and metrics for the posterior redshift distribution. A commonly used metric for point prediction performance is the outlier fraction. Outliers are typically defined as objects for which the normalized offsets are greater than 15\% from the truth,

\begin{equation}
\bigg|\frac{z_{\rm phot} - z_{\rm spec}}{1 + z_{\rm spec}}\bigg| > 0.15,
\end{equation}

giving the outlier fraction, $\eta$, as

\begin{equation}
\eta = n_{\rm outliers}/n_{\rm total}
\end{equation}

We also look at the standard deviation estimated from the normalized median absolute deviation, because it is less sensitive to outliers than the typical definition: 

\begin{equation}
\sigma_{\rm nmad} = 1.48 \times {\rm median} \bigg|\frac{z_{\rm phot} - z_{\rm spec}}{1 + z_{\rm spec}} \bigg|
\end{equation}

and the bias

\begin{equation}
\beta =  {\rm median} \left( \frac{z_{\rm phot} - z_{\rm spec}}{1 + z_{spec}} \right)
\end{equation}

Because $\sigma_{\rm nmad}$ will be impacted by $\beta$ we can also compute an unbiased NMAD estimator as

\begin{equation}
\sigma_{\rm nmad, unbiased} = 1.48 \times {\rm median} \bigg|\frac{z_{\rm phot} - z_{\rm spec} - {\rm median} ( z_{\rm phot} - z_{\rm spec} )}{1 + z_{\rm spec}}  \bigg|
\end{equation}

In [ ]:
gal=input_lp['string_input']=='galaxy'
gal&=output['ZSPEC']>0.002
plt.scatter(output['ZSPEC'][gal],output['Z_BEST'][gal],s=0.2)
plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")

In [ ]:
agn=input_lp['string_input']=='x'
agn|=input_lp['string_input']=='x+broad'
agn&=output['ZSPEC']>0.002
plt.scatter(output['ZSPEC'][agn],output['ZQ_BEST'][agn],s=0.2)
plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")

In [ ]:
plt.scatter(output['ZSPEC'][agn],output['ZQ_BEST'][agn],s=0.2)
plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")

In [ ]:
plt.scatter(output_agn['ZSPEC'][agn],output_agn['ZQ_BEST'][agn],s=0.2)
plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")
plt.title("X-ray AGN with the QSO templates but AGN absolute magnitude priors")

In [ ]:
broad=input_lp['string_input']=='broad'
broad|=input_lp['string_input']=='x+broad'
broad&=output['ZSPEC']>0.002
plt.scatter(output['ZSPEC'][broad],output['ZQ_BEST'][broad],s=0.2)
plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")
plt.title("Broadline AGN with the QSO templates")

In [ ]:
plt.scatter(output_agn['ZSPEC'][broad],output_agn['ZQ_BEST'][broad],s=0.2)
plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")

In [ ]:
plt.scatter(output['ZSPEC'][agn],output['Z_BEST'][agn],s=0.2)
plt.xlabel("$z_{spec}$")
plt.ylabel("$z_{phot}$")
plt.title("X-ray selected AGN with the galaxy templates")

In [ ]:
plt.hist(output['ZSPEC'][gal],bins=20,density=True,label='Galaxies',alpha=0.5)
plt.hist(output['ZSPEC'][agn],bins=20,density=True,label='X-ray AGN',alpha=0.5)
plt.hist(output['ZSPEC'][broad],bins=20,density=True,label='Broadline AGN',alpha=0.5)
plt.legend()
plt.xlabel('$z_{spec}$')

In [ ]:
delz_norm=(output['Z_BEST']-output['ZSPEC'])/(1+output['ZSPEC'])
delz_norm=(output['Z_BEST']-output['ZSPEC'])/(1+output['ZSPEC'])-np.median(delz_norm)
delzq_norm=(output['ZQ_BEST']-output['ZSPEC'])/(1+output['ZSPEC'])
delzq_norm=(output['ZQ_BEST']-output['ZSPEC'])/(1+output['ZSPEC'])-np.median(delzq_norm)
delzq_norm_agn=(output_agn['ZQ_BEST']-output_agn['ZSPEC'])/(1+output_agn['ZSPEC'])
delzq_norm_agn=(output_agn['ZQ_BEST']-output_agn['ZSPEC'])/(1+output_agn['ZSPEC'])-np.median(delzq_norm_agn)
print(f"""Galaxy $\sigma_{{nmad}}$ for Z_BEST: {1.48*np.median(np.abs(delz_norm[gal])):.4f}
Galaxy $\sigma_{{nmad}}$ for ZQ_BEST: {1.48*np.median(np.abs(delzq_norm[gal])):.4f}    
X-ray AGN $\sigma_{{nmad}}$ for Z_BEST: {1.48*np.median(np.abs(delz_norm[agn])):.4f}
Broadline AGN $\sigma_{{nmad}}$ for Z_BEST: {1.48*np.median(np.abs(delz_norm[broad])):.4f}
X-ray AGN $\sigma_{{nmad}}$ for ZQ_BEST with 'QSO' settings: {1.48*np.median(np.abs(delzq_norm[agn])):.4f}    
Broadline AGN $\sigma_{{nmad}}$ for ZQ_BEST with 'QSO' settings: {1.48*np.median(np.abs(delzq_norm[broad])):.4f}
X-ray AGN $\sigma_{{nmad}}$ for ZQ_BEST with 'AGN' settings: {1.48*np.median(np.abs(delzq_norm_agn[agn])):.4f}    
Broadline AGN $\sigma_{{nmad}}$ for ZQ_BEST with 'AGN' settings: {1.48*np.median(np.abs(delzq_norm_agn[broad])):.4f} 
    """)

In [ ]:
plt.hist(output['CHI_QSO']-output['CHI_BEST'],bins=50,range=[-500,500])
plt.xlabel('CHI_QSO-CHI_BEST')
plt.yscale('log')

In [ ]:
for s in np.unique(input_lp['string_input']):
    m=input_lp['string_input']==s    
    plt.hist(output['CHI_QSO'][m]-output['CHI_BEST'][m],bins=50, label=s, histtype='step', alpha=1., range=[-100,100],density=True)
plt.xlabel('CHI_QSO-CHI_BEST')
plt.title('QSO config')
plt.legend()
plt.yscale('log')

In [ ]:
for s in np.unique(input_lp['string_input']):
    m=input_lp['string_input']==s    
    plt.hist(output_agn['CHI_QSO'][m]-output['CHI_BEST'][m],bins=50, label=s, histtype='step', alpha=1., range=[-100,100],density=True)
plt.xlabel('CHI_QSO-CHI_BEST')
plt.title('AGN config')
plt.legend()
plt.yscale('log')

In [ ]:
chi_cut=np.linspace(-100,100,20)
chi_diff=output['CHI_QSO']-output['CHI_BEST']
mids=np.array([])
fracs=np.array([])
tpr=np.array([])
fpr=np.array([])
for n,c in enumerate(chi_cut[:-1]):
    mids = np.append(mids, np.sum(chi_cut[n:n+1])/2)
    in_bin=chi_diff>c
    in_bin&=chi_diff<=chi_cut[n+1]
    fracs = np.append(fracs, np.sum(input_lp['string_input'][in_bin]=='galaxy')/np.sum(in_bin))

    # Calculate the true positive rate and false positive rate to investigate classification power
    tp=np.sum(chi_diff[input_lp['string_input']!='galaxy']<c)
    tn=np.sum(chi_diff[input_lp['string_input']=='galaxy']>=c)
    fp=np.sum(chi_diff[input_lp['string_input']=='galaxy']<c)
    fn=np.sum(chi_diff[input_lp['string_input']!='galaxy']>=c)
    tpr=np.append(tpr,tp/(tp+fn))
    fpr=np.append(fpr,fp/(fp+tn))

plt.plot(mids, fracs)   
plt.xlabel('CHI_QSO-CHI_BEST')
plt.ylabel('$f_{gal}$')

In [ ]:
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1])
plt.xlabel('False postive rate')
plt.ylabel('True positive rate')
plt.xlim([0,1])
plt.ylim([0,1])

## Look at some example SEDs

What are the differences between the different samples?

How clearly does the $\chi^2$ difference between the best QSO and best galaxy relate to type?

In [ ]:
#gal
from os import listdir
from os.path import isfile, join

listname = [f for f in listdir("save_spec") if isfile(join("save_spec", f))]
# Lets just look at the top 10
for namefile in listname[:1]:
    lp.plotspec("save_spec/" + str(namefile))

In [ ]:
# The first ten objects of type x
for i in output['IDENT'][input_lp['string_input']=='x'][:10]:
    lp.plotspec(f"save_spec/Id{i}.spec")

In [ ]:
# The first ten objects of type x
for i in output['IDENT'][input_lp['string_input']=='galaxy'][:10]:
    lp.plotspec(f"save_spec/Id{i}.spec")

In [ ]:
# The first ten objects of type x
for i in output['IDENT'][input_lp['string_input']=='broad'][:10]:
    lp.plotspec(f"save_spec/Id{i}.spec")